In [97]:
import numpy as np
import pandas as pd

import openpyxl

In [98]:
# Variables
international_file = '~/Development/business-analysis/Data/International.xlsx'
domestic_file = '~/Development/business-analysis/Data/Domestic.xlsx'
order_file = '~/Development/business-analysis/Data/open_order.xlsx'
supplier_file = '~/Development/business-analysis/Data/suppliers.xlsx'

export_location = '~/Desktop/report.xlsx'

files = [international, domestic, orders, suppliers]

# for later
max_total_stock = 12
max_international_stock = 9 #stock with 12 months of stock

# weighting for avg monthly international usage
one_month = 0.6
three_month = 0.3
six_month = 0.1

In [ ]:
# import files

In [43]:
international = pd.read_excel(international_file)
domestic = pd.read_excel(domestic_file)
orders = pd.read_excel(order_file)
suppliers = pd.read_excel(supplier_file)

In [44]:
for file in files:
    print(file.head())

   part_number  inventory  monthly_avg(one)  monthly_avg(three)  \
0            1       1350                60           58.333333   
1            2       1900                80          173.333333   
2            3      49300              1000          333.333333   
3            4      33443              5000         8666.666667   
4            5      67063             10000         7933.333333   

   monthly_avg(six)  
0         56.666667  
1        100.000000  
2        250.000000  
3       6833.333333  
4       6800.000000  
   part_number  inventory  one_mo_usage
0          105     7400.0           0.0
1          106        0.0        1000.0
2          108        0.0           0.0
3          109    44000.0           0.0
4          110    13000.0           0.0
   part_number  On Order
0          105  100000.0
1          106   50000.0
2          108  150000.0
3          109  100000.0
4          110  100000.0
   part_number      d_f
0            1  Foreign
1            2  Foreign
2  

In [51]:
# merge files

In [45]:
report = domestic.merge(international, on='part_number')
report.set_index('part_number')
report.head()

,part_number,inventory_x,one_mo_usage,inventory_y,monthly_avg(one),monthly_avg(three),monthly_avg(six)
0,105,7400.0,0.0,25800,6200,6290.000000,7345.000000
1,106,0.0,1000.0,57354,4500,2733.333333,2866.666667
2,108,0.0,0.0,96179,15000,12166.666667,13000.000000
3,109,44000.0,0.0,72992,8000,8500.000000,9416.666667
4,110,13000.0,0.0,74000,8000,6333.333333,7166.666667


In [52]:
# clean up data and combine usage

In [46]:
report.columns = ['part_number', 'domestic_inventory', 'domestic_usage', 'international_inventory', 'international_usage_1','international_usage_3','international_usage_6']

In [47]:
report.head()

,part_number,domestic_inventory,domestic_usage,international_inventory,international_usage_1,international_usage_3,international_usage_6
0,105,7400.0,0.0,25800,6200,6290.000000,7345.000000
1,106,0.0,1000.0,57354,4500,2733.333333,2866.666667
2,108,0.0,0.0,96179,15000,12166.666667,13000.000000
3,109,44000.0,0.0,72992,8000,8500.000000,9416.666667
4,110,13000.0,0.0,74000,8000,6333.333333,7166.666667


In [48]:
report['international_usage'] = (report['international_usage_1'] * one_month) + \
    (report['international_usage_3'] * three_month) + \
    (report['international_usage_6'] * six_month)

In [49]:
report = report.drop('international_usage_1', axis=1) 
report = report.drop('international_usage_3', axis=1)
report = report.drop('international_usage_6', axis=1)
report.head()

,part_number,domestic_inventory,domestic_usage,international_inventory,international_usage
0,105,7400.0,0.0,25800,6341.500000
1,106,0.0,1000.0,57354,3806.666667
2,108,0.0,0.0,96179,13950.000000
3,109,44000.0,0.0,72992,8291.666667
4,110,13000.0,0.0,74000,7416.666667


In [53]:
# add the column needed

In [50]:
report['total_inventory'] = report['domestic_inventory'] + report['international_inventory']
report['total_usage'] = report['domestic_usage'] + report['international_usage']


In [54]:
# Create months inventory

In [55]:
report['total_inventory_months'] = report['total_inventory'] / report['total_usage']

In [57]:
# What do we need to order

In [60]:
report['external_orders'] = np.where(report['total_inventory_months'] < 6, 'order', 'wait')
report.head()

,part_number,domestic_inventory,domestic_usage,international_inventory,international_usage,total_inventory,total_usage,total_inventory_months,external_orders
0,105,7400.0,0.0,25800,6341.500000,33200.0,6341.500000,5.235354,order
1,106,0.0,1000.0,57354,3806.666667,57354.0,4806.666667,11.932178,wait
2,108,0.0,0.0,96179,13950.000000,96179.0,13950.000000,6.894552,wait
3,109,44000.0,0.0,72992,8291.666667,116992.0,8291.666667,14.109588,wait
4,110,13000.0,0.0,74000,7416.666667,87000.0,7416.666667,11.730337,wait


In [61]:
# ...now we need to use the lead time rather than an arb 6 months

In [63]:
suppliers['lead_time'] = np.where(suppliers['d_f'] == 'Foreign', 6, 1)

In [67]:
report = report.merge(suppliers, on='part_number')
report = report.drop('d_f', axis=1)

In [73]:
report['external_orders'] = np.where(report['total_inventory_months'] <= (1 + report['lead_time']), 'order', 'wait')
report['order_quantity'] = np.where(report['external_orders'] == 'order', max_total_stock * report['total_usage'] - report['total_inventory'],0)
report.head()

,part_number,domestic_inventory,domestic_usage,international_inventory,international_usage,total_inventory,total_usage,total_inventory_months,external_orders,lead_time,domestic_excess_stock,order_quantity
0,105,7400.0,0.0,25800,6341.500000,33200.0,6341.500000,5.235354,order,6,7400.0,42898.0
1,106,0.0,1000.0,57354,3806.666667,57354.0,4806.666667,11.932178,wait,6,-6000.0,0.0
2,108,0.0,0.0,96179,13950.000000,96179.0,13950.000000,6.894552,wait,1,0.0,0.0
3,109,44000.0,0.0,72992,8291.666667,116992.0,8291.666667,14.109588,wait,6,44000.0,0.0
4,110,13000.0,0.0,74000,7416.666667,87000.0,7416.666667,11.730337,wait,6,13000.0,0.0


In [69]:
# Now let's look at transfers

In [77]:
report['domestic_excess_stock'] = report.domestic_inventory - (report.lead_time * report.domestic_usage)
report['domestic_excess_stock'] = report['domestic_excess_stock'].clip(lower=0)
report['domestic_excess_stock'] = report['domestic_excess_stock'].fillna(0)

In [74]:
report['international_need'] = report.international_usage * max_international_stock

In [84]:
#ship if available
report['transfer'] = report[['domestic_excess_stock','international_need']].min(axis=1)

In [85]:
report.head()

,part_number,domestic_inventory,domestic_usage,international_inventory,international_usage,total_inventory,total_usage,total_inventory_months,external_orders,lead_time,domestic_excess_stock,order_quantity,international_need,transfer
0,105,7400.0,0.0,25800,6341.500000,33200.0,6341.500000,5.235354,order,6,7400.0,42898.0,57073.5,7400.0
1,106,0.0,1000.0,57354,3806.666667,57354.0,4806.666667,11.932178,wait,6,0.0,0.0,34260.0,0.0
2,108,0.0,0.0,96179,13950.000000,96179.0,13950.000000,6.894552,wait,1,0.0,0.0,125550.0,0.0
3,109,44000.0,0.0,72992,8291.666667,116992.0,8291.666667,14.109588,wait,6,44000.0,0.0,74625.0,44000.0
4,110,13000.0,0.0,74000,7416.666667,87000.0,7416.666667,11.730337,wait,6,13000.0,0.0,66750.0,13000.0


In [87]:
report = report.drop(['domestic_excess_stock', 'international_need'], axis=1)

In [88]:
# find problems
report['Urgent'] = np.where(report['total_inventory_months'] < report['lead_time'], 'Urgent', ' ')

In [89]:
report.head()

,part_number,domestic_inventory,domestic_usage,international_inventory,international_usage,total_inventory,total_usage,total_inventory_months,external_orders,lead_time,order_quantity,transfer,Urgent
0,105,7400.0,0.0,25800,6341.500000,33200.0,6341.500000,5.235354,order,6,42898.0,7400.0,Urgent
1,106,0.0,1000.0,57354,3806.666667,57354.0,4806.666667,11.932178,wait,6,0.0,0.0,
2,108,0.0,0.0,96179,13950.000000,96179.0,13950.000000,6.894552,wait,1,0.0,0.0,
3,109,44000.0,0.0,72992,8291.666667,116992.0,8291.666667,14.109588,wait,6,0.0,44000.0,
4,110,13000.0,0.0,74000,7416.666667,87000.0,7416.666667,11.730337,wait,6,0.0,13000.0,


In [99]:
report.to_excel(export_location)

In [ ]:
# need openpyxl